## Image Resizing

In [2]:
import os
import glob
from PIL import Image

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/Extracted_Processed/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

916  images to be resized.
916  images resized.
가위 이미지 resize 완료!


In [3]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/Extracted_Processed/rock"

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
resize_images(image_dir_path)
print("바위 이미지 resize 완료!")

910  images to be resized.
910  images resized.
바위 이미지 resize 완료!


In [4]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/Extracted_Processed/paper"

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
resize_images(image_dir_path)
print("보 이미지 resize 완료!")

916  images to be resized.
916  images resized.
보 이미지 resize 완료!


## Image Labeling

In [5]:
import numpy as np

def load_data(img_path, number_of_data=2742):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/Extracted_Processed"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 2742 입니다.
x_train shape: (2742, 28, 28, 3)
y_train shape: (2742,)


In [98]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(15, (2,2), activation='relu', input_shape=(28,28,3), padding='same'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(20, (2,2), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(5, (2,2), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(9, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()

Model에 추가된 Layer 개수:  10
Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 28, 28, 15)        195       
_________________________________________________________________
batch_normalization_36 (Batc (None, 28, 28, 15)        60        
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 14, 14, 15)        0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 14, 14, 20)        1220      
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 7, 7, 20)          0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 7, 7, 5)           405       
_________________________________________________________________
max_pooling2d_80 (MaxPooling

In [99]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=8)

Epoch 1/8
86/86 [==============================] - 1s 4ms/step - loss: 1.0973 - accuracy: 0.4020
Epoch 2/8
86/86 [==============================] - 0s 3ms/step - loss: 0.8115 - accuracy: 0.6725
Epoch 3/8
86/86 [==============================] - 0s 3ms/step - loss: 0.5410 - accuracy: 0.7760
Epoch 4/8
86/86 [==============================] - 0s 4ms/step - loss: 0.3741 - accuracy: 0.8646
Epoch 5/8
86/86 [==============================] - 0s 3ms/step - loss: 0.2984 - accuracy: 0.8973
Epoch 6/8
86/86 [==============================] - 0s 3ms/step - loss: 0.2299 - accuracy: 0.9200
Epoch 7/8
86/86 [==============================] - 0s 3ms/step - loss: 0.1850 - accuracy: 0.9424
Epoch 8/8
86/86 [==============================] - 0s 3ms/step - loss: 0.1609 - accuracy: 0.9508


##  Load testset, Preprocessing

In [56]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
import os	
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/test/scissor"
resize_images(image_dir_path)
print("가위 이미지 resize 완료!")
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/test/rock"
resize_images(image_dir_path)
print("바위 이미지 resize 완료!")
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/test/paper"
resize_images(image_dir_path)
print("보 이미지 resize 완료!")

test_image_path = os.getenv("HOME") + "/aiffel/rock_scissors_paper/test"
(x_test, y_test)=load_data(test_image_path, number_of_data=327)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화, Training set 과 같은 처리

109  images to be resized.
109  images resized.
가위 이미지 resize 완료!
109  images to be resized.
109  images resized.
바위 이미지 resize 완료!
109  images to be resized.
109  images resized.
보 이미지 resize 완료!
학습데이터(x_train)의 이미지 개수는 327 입니다.


In [100]:
model.evaluate(x_test_norm, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.7899 - accuracy: 0.6330


[0.7898632287979126, 0.6330274939537048]

**0.63의 정확도를 보였다.**